In [1]:
print("\n\nம ஹா ம்ரி த் யு ஞ் ஜ யா\n\n")



ம ஹா ம்ரி த் யு ஞ் ஜ யா




In [2]:
from scipy import stats

In [3]:
#import tensorflow as tf

import pickle
import pandas as pd
import os
from matplotlib import pyplot
from pathlib import Path
#from tensorflow.contrib.rnn import LSTMCell, GRUCell
import numpy as np
import time, uuid
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
#

In [4]:
def Box_filter(values, kernel_size=3):
    """
    :return: The list of filtered average
    """
    filter_values = np.cumsum(values, dtype=float)

    filter_values[kernel_size:] = filter_values[kernel_size:] - filter_values[:-kernel_size]
    filter_values[kernel_size:] = filter_values[kernel_size:] / kernel_size

    for i in range(1, kernel_size):
        filter_values[i] /= i + 1

    return filter_values

In [5]:
def extrapolate_next(values):
    """
    Extrapolates the next value by sum up the slope of the last value with previous values.
    :param values: a list or numpy array of time-series
    :return: the next value of time-series
    """

    last_value = values[-1]
    slope = [(last_value - v) / i for (i, v) in enumerate(values[::-1])]
    slope[0] = 0
    next_values = last_value + np.cumsum(slope)

    return next_values

In [6]:
def merge_series(values, extend_num=5, forward=5):

    next_value = extrapolate_next(values)[forward]
    extension = [next_value] * extend_num

    if isinstance(values, list):
        merge_values = values + extension
    else:
        merge_values = np.append(values, extension)
    return merge_values

In [7]:
class Silency(object):
    def __init__(self, amp_window_size, series_window_size, score_window_size):
        self.amp_window_size = amp_window_size
        self.series_window_size = series_window_size
        self.score_window_size = score_window_size

    def frequency_domain_map(self, values):
        freq = np.fft.fft(values)
        mag = np.sqrt(freq.real ** 2 + freq.imag ** 2)
        spectral_residual = np.exp(np.log(mag) - Box_filter(np.log(mag), self.amp_window_size))

        freq.real = freq.real * spectral_residual / mag
        freq.imag = freq.imag * spectral_residual / mag

        freqmap = np.fft.ifft(freq)
        freq_map = np.sqrt(freqmap.real ** 2 + freqmap.imag ** 2)
        return freq_map

    def generate_anomaly_score(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        ave_filter = Box_filter(mag, self.score_window_size)        
        score = (mag - ave_filter) / ave_filter 
        
        return score

    def generate_anomaly_score2(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        return mag

    def generate_anomaly_score_(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        ave_filter = Box_filter(mag, self.score_window_size)
        
        ave_filter1 = Box_filter(ave_filter, 12)
        ave_filter2 = Box_filter(ave_filter1, 8)
        ave_filter = Box_filter(ave_filter2, 6)
        
        score = (mag - ave_filter) / ave_filter
        
        return score
    

In [8]:
def box_filter(ts, n) :
    ret = np.cumsum(ts, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    ret[n:] = ret[n:] / n

    for i in range(1, n):
        ret[i] /= i + 1

    return ret


# Entire A4 folder

In [9]:
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'


In [10]:
import os
import glob

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

In [11]:
files

['real_59.csv',
 'real_65.csv',
 'A3Benchmark-TS12.csv',
 'synthetic_85.csv',
 'synthetic_91.csv',
 'A4Benchmark-TS99.csv',
 'A4Benchmark-TS72.csv',
 'synthetic_46.csv',
 'synthetic_52.csv',
 'A4Benchmark-TS66.csv',
 'synthetic_53.csv',
 'A4Benchmark-TS67.csv',
 'A4Benchmark-TS73.csv',
 'synthetic_47.csv',
 'A4Benchmark-TS98.csv',
 'synthetic_90.csv',
 'synthetic_84.csv',
 'A3Benchmark-TS13.csv',
 'real_64.csv',
 'real_58.csv',
 'real_66.csv',
 'A3Benchmark-TS11.csv',
 'A3Benchmark-TS39.csv',
 'synthetic_92.csv',
 'synthetic_86.csv',
 'A4Benchmark-TS65.csv',
 'synthetic_51.csv',
 'synthetic_45.csv',
 'A4Benchmark-TS71.csv',
 'A4Benchmark-TS59.csv',
 'synthetic_79.csv',
 'synthetic_78.csv',
 'synthetic_100.csv',
 'A4Benchmark-TS58.csv',
 'synthetic_44.csv',
 'A4Benchmark-TS70.csv',
 'A4Benchmark-TS64.csv',
 'synthetic_50.csv',
 'synthetic_87.csv',
 'synthetic_93.csv',
 'A3Benchmark-TS38.csv',
 'A3Benchmark-TS10.csv',
 'real_67.csv',
 'real_63.csv',
 'A3Benchmark-TS28.csv',
 'A3Benchmark

In [12]:
# less than period
amp_window_size=4
# (maybe) as same as period
series_window_size=4
# a number enough larger than period
score_window_size=16

spec = Silency(amp_window_size, series_window_size, score_window_size)

In [13]:

prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)

    test_signal = df.value.values 

    score = spec.generate_anomaly_score_(test_signal)

    index_changes = np.where(score > np.percentile(score, 99))[0]

    cutoff = np.percentile(score, 99)

    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
#    pred_anom = np.zeros(len(test_signal))
#    for i in range(int(len(test_signal))):
#        for j in index_changes:
#            pred_anom[j] = 1

    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )


/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


In [14]:
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

(0.40723272960410317, 0.8852972149525491, 0.5047092172414838)

In [15]:
import time

# EXPERIMENTS: 

In [16]:
# less than period
amp_window_size=3
# (maybe) as same as period
series_window_size=3
# a number enough larger than period
score_window_size=32

spec = Silency(amp_window_size, series_window_size, score_window_size)

# 1

# fixed threshold '4'

# BEST parameter set, F1_score = 74.57%

In [52]:
path='/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/A1Benchmark/'
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

67

### Detrending tried only for (FULL DATA) this experiment

In [53]:
st = time.time()
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_sig = df.value.values 
    
    ma = box_filter(test_sig, n=24)
    test_signal = test_sig - ma # detrend
        
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
        print(file, f1_score(df.anomaly.values, pred_anom))
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )
        
        print(file, f1_score(df.is_anomaly.values, pred_anom))
print("Time taken =", (time.time()-st), "seconds")
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list), len(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


real_59.csv 0.0
real_65.csv 0.09523809523809523
real_64.csv 0.0
real_58.csv 0.04444444444444444
real_66.csv 0.25
real_67.csv 0.08
real_63.csv 0.0
real_62.csv 0.6153846153846153
real_60.csv 0.3703703703703703
real_48.csv 0.0
real_49.csv 0.5
real_61.csv 0.0
real_12.csv 0.5714285714285715
real_5.csv 0.28571428571428575
real_4.csv 0.4000000000000001
real_13.csv 0.5
real_39.csv 0.5263157894736842
real_11.csv 0.2727272727272727
real_6.csv 0.4
real_7.csv 0.0
real_10.csv 0.0
real_38.csv 0.3157894736842105
real_14.csv 0.13333333333333336
real_28.csv 0.06818181818181818
real_3.csv 0.26086956521739135
real_2.csv 0.3636363636363636
real_29.csv 0.6666666666666666
real_15.csv 0.588235294117647
real_17.csv 0.04273504273504274
real_1.csv 1.0
real_16.csv 0.8
real_33.csv 0.3333333333333333
real_27.csv 0.6666666666666666
real_26.csv 0.0
real_32.csv 0.21428571428571427
real_18.csv 0.5
real_24.csv 0.2857142857142857
real_30.csv 0.26666666666666666
real_31.csv 0.13333333333333333
real_25.csv 0.0416666666666

(0.47181184828243655, 0.3215923447859204, 0.32551326227431154, 67)

In [50]:
print("Time taken =", (time.time()-st), "seconds")
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list), len(f1score_list)

Time taken = 20.755525827407837 seconds


(0.8198337983720256, 0.8005761167654072, 0.7790611637889208, 367)

In [ ]:
# Individual Fscore of Yahoo's A1, A2, A3, A4
#A1 = 32.55%
#A2 = 78.09%
#A3 = 95.41%
#A4 = 90.59%

In [46]:
(67*(32.55)+7809+9541+9059)/367

77.90149863760217

### ### Detrending tried only for (HALF DATA) this experiment  

In [40]:
path='/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/A4Benchmark/'
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

In [41]:
st = time.time()
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    #test_signal = df.value.values 
    test_sig = df.value.values 
    
    ma = box_filter(test_sig, n=24)
    test_signal = test_sig - ma # detrend
    
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

print("Time taken =", (time.time()-st), "seconds")
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classificatio

Time taken = 2.5962510108947754 seconds


(0.8142532762423771, 0.7594252731911294, 0.7512426147356498)

In [42]:
len(f1score_list)

367

In [43]:
# Individual Fscore of Yahoo's A1, A2, A3, A4
#A1 = 29.07%
#A2 = 78.65%
#A3 = 92.05%
#A4 = 85.55

In [48]:
((67*29.07)+7865+9205+8555)/367

75.1299455040872

# 2

# Enhanced SR with no averaging/filtering of saliency map
# and
# cutoff = 7 * np.mean(score)

def generate_anomaly_score(self, values, type="avg"):

        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        
        mag = self.frequency_domain_map(extended_series)[: len(values)]
        
        return mag

In [145]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.6509540021083907, 0.7779652423245614, 0.6508797391830556)

In [146]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.675088970218398, 0.7560466291248261, 0.6570270138255867)

# 3

# Enhanced SR with averaging/filtering of saliency map
# cutoff = 2.8

In [147]:
#def generate_anomaly_score(self, values, type="avg"):
#    extended_series = merge_series(values, self.series_window_size, self.series_window_size)
#    mag = self.frequency_domain_map(extended_series)[: len(values)]
#    ave_filter = Box_filter(mag, self.score_window_size)
#    score = (mag - ave_filter) / ave_filter
#    return score

In [185]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.5968055594659185, 0.8690790440124405, 0.6590783629749551)

In [186]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.6589798720628981, 0.8312036686714798, 0.6940316418644611)

# Half of the data [ TEST ]

In [88]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2): , : ]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.784573328578778, 0.7552290545053432, 0.7320365590665248)

In [14]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)

    test_signal = df.value.values

    score = spec.generate_anomaly_score(test_signal)

    index_changes = np.where(score > np.percentile(score, 98))[0]

    cutoff = np.percentile(score, 98)

    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)

    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.21379291921598106, 0.9024549278672852, 0.3206594749799114)

In [15]:
len(files)

367

In [16]:
cnt=0
for file in files:
    if "Benchmark" in file:
        cnt = cnt + 1
print(cnt)

200
